# Interactive Deterrence Playground

Use the sliders below to choose parameters and see how total system wealth evolves. Each configuration is averaged across multiple Monte Carlo runs (default: 10; adjustable via the “MC runs” slider) so the curves reflect policy rather than random noise. The chart updates automatically whenever you release a slider.

*Note: Nuclear weapons are not modeled yet—this focuses on conventional deterrence and bargaining.*


In [1]:
import importlib
import contextlib
import io

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

import deterrence_simulation as ds

plt.style.use('seaborn-v0_8')



In [2]:
def run_simulation(
    num_countries: int,
    num_rounds: int,
    attack_cost: float,
    failed_attack_cost: float,
    defender_defense_loss: float,
    bargain_ev: float,
    trials: int = 10,
    base_seed: int = 42,
):
    """Run multiple simulation trials and return averages for plotting."""
    original_params = {
        'NUM_COUNTRIES': ds.NUM_COUNTRIES,
        'ATTACK_COST_PERCENTAGE': ds.ATTACK_COST_PERCENTAGE,
        'FAILED_ATTACK_COST_PERCENTAGE': ds.FAILED_ATTACK_COST_PERCENTAGE,
        'ATTACK_SUCCESS_DISCOUNT_CAP': ds.ATTACK_SUCCESS_DISCOUNT_CAP,
        'DEFENDER_DEFENSE_LOSS_PERCENTAGE': ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE,
        'BARGAIN_EV_PERCENTAGE': ds.BARGAIN_EV_PERCENTAGE,
    }

    ds.NUM_COUNTRIES = num_countries
    ds.ATTACK_COST_PERCENTAGE = attack_cost
    ds.FAILED_ATTACK_COST_PERCENTAGE = failed_attack_cost
    ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE = defender_defense_loss
    ds.BARGAIN_EV_PERCENTAGE = bargain_ev

    round_count = num_rounds + 1
    totals_matrix = np.zeros((trials, round_count))
    active_matrix = np.zeros((trials, round_count))
    bargains_matrix = np.zeros((trials, num_rounds))
    attacks_matrix = np.zeros((trials, num_rounds))

    # Track sample country trajectories and actions from first trial
    sample_country_values = None
    country_order = None
    sample_country_actions = {}

    try:
        for t in range(trials):
            seed = base_seed + t
            sim = ds.DeterrenceSimulation(num_countries=num_countries, seed=seed)
            totals_matrix[t, 0] = sim.get_total_value()
            active_matrix[t, 0] = sum(1 for c in sim.countries.values() if c.is_active)

            # Capture sample data from first trial
            if t == 0:
                country_order = list(sim.countries.keys())
                sample_country_values = np.zeros((round_count, len(country_order)))
                sample_country_values[0, :] = [sim.countries[cid].private_value for cid in country_order]
                for cid in country_order:
                    sample_country_actions[cid] = {'attacks': 0, 'bargains': 0}

            for round_idx in range(num_rounds):
                with contextlib.redirect_stdout(io.StringIO()):
                    results = sim.run_round()
                totals_matrix[t, round_idx + 1] = sim.get_total_value()
                active_matrix[t, round_idx + 1] = sum(1 for c in sim.countries.values() if c.is_active)
                bargains_matrix[t, round_idx] = sum(1 for r in results if r['outcome'] == 'bargain')
                attacks_matrix[t, round_idx] = sum(1 for r in results if r['outcome'] != 'bargain')

                # Track sample country values and actions from first trial
                if t == 0:
                    sample_country_values[round_idx + 1, :] = [sim.countries[cid].private_value if cid in sim.countries and sim.countries[cid].is_active else 0.0 for cid in country_order]
                    for r in results:
                        initiator = r.get('initiator')
                        target = r.get('target')
                        if initiator is not None and initiator not in sample_country_actions:
                            sample_country_actions[initiator] = {'attacks': 0, 'bargains': 0}
                        if target is not None and target not in sample_country_actions:
                            sample_country_actions[target] = {'attacks': 0, 'bargains': 0}

                        if r['outcome'] == 'bargain':
                            if initiator is not None:
                                sample_country_actions[initiator]['bargains'] += 1
                            if target is not None:
                                sample_country_actions[target]['bargains'] += 1
                        else:
                            if initiator is not None:
                                sample_country_actions[initiator]['attacks'] += 1
    finally:
        ds.NUM_COUNTRIES = original_params['NUM_COUNTRIES']
        ds.ATTACK_COST_PERCENTAGE = original_params['ATTACK_COST_PERCENTAGE']
        ds.FAILED_ATTACK_COST_PERCENTAGE = original_params['FAILED_ATTACK_COST_PERCENTAGE']
        ds.ATTACK_SUCCESS_DISCOUNT_CAP = original_params['ATTACK_SUCCESS_DISCOUNT_CAP']
        ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE = original_params['DEFENDER_DEFENSE_LOSS_PERCENTAGE']
        ds.BARGAIN_EV_PERCENTAGE = original_params['BARGAIN_EV_PERCENTAGE']

    rounds = list(range(round_count))
    avg_totals = totals_matrix.mean(axis=0)
    avg_active = active_matrix.mean(axis=0)
    avg_bargains = bargains_matrix.mean(axis=0)
    avg_attacks = attacks_matrix.mean(axis=0)

    pct_change = np.zeros_like(avg_totals)
    pct_change[1:] = np.where(
        avg_totals[:-1] > 0,
        (avg_totals[1:] - avg_totals[:-1]) / avg_totals[:-1] * 100,
        0,
    )

    return rounds, avg_totals, pct_change, avg_active, avg_bargains, avg_attacks, sample_country_values, country_order, sample_country_actions



In [3]:
def plot_totals(num_countries, num_rounds, attack_cost, failed_attack_cost, defender_defense_loss, bargain_ev, mc_trials):
    rounds, totals, pct_change, active_counts, bargains, attacks, country_traces, country_order, country_actions = run_simulation(
        num_countries=int(num_countries),
        num_rounds=int(num_rounds),
        attack_cost=attack_cost,
        failed_attack_cost=failed_attack_cost,
        defender_defense_loss=defender_defense_loss,
        bargain_ev=bargain_ev,
        trials=int(mc_trials),
    )

    fig, axes = plt.subplots(3, 1, figsize=(10, 13), sharex=True,
                             gridspec_kw={'height_ratios': [2, 1, 1]})
    ax_total, ax_pct, ax_active = axes

    ax_total.plot(rounds, totals, marker='o', linewidth=2, color='#004488')
    ax_total.set_ylabel('Average Total System Value', fontsize=12, fontweight='bold')
    ax_total.set_title(f'Total Wealth Over Time (averaged over {int(mc_trials)} runs)', fontsize=14, fontweight='bold')
    ax_total.grid(True, alpha=0.3)

    summary_lines = [f"Round {r}: {b:.1f} bargains / {a:.1f} attacks" for r, b, a in zip(rounds[1:], bargains, attacks)]
    annotation = "\n".join(summary_lines) if summary_lines else "No rounds run"
    ax_total.text(1.02, 0.5, annotation, transform=ax_total.transAxes, va='center', fontsize=10,
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    ax_pct.bar(rounds[1:], pct_change[1:], color='#E67E22', alpha=0.8)
    ax_pct.axhline(0, color='black', linewidth=1)
    ax_pct.set_ylabel('% Change vs Prev Round', fontsize=12, fontweight='bold')
    ax_pct.set_title('Percentage Change in Total Wealth', fontsize=12, fontweight='bold')
    ax_pct.grid(True, alpha=0.3)

    ax_active.plot(rounds, active_counts, marker='s', linewidth=2, color='#2E8B57')
    ax_active.set_xlabel('Round', fontsize=12, fontweight='bold')
    ax_active.set_ylabel('Avg Active Countries', fontsize=12, fontweight='bold')
    ax_active.set_title('Average Countries Remaining', fontsize=12, fontweight='bold')
    ax_active.grid(True, alpha=0.3)
    ax_active.set_ylim(bottom=0)

    plt.tight_layout()
    plt.show()

    if country_traces is not None and country_order is not None:
        rounds_range = list(range(country_traces.shape[0]))
        num_countries_to_plot = country_traces.shape[1]
        cmap = plt.get_cmap('tab20')

        fig2, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True)
        ax_log, ax_linear = axes

        for idx in range(num_countries_to_plot):
            line = country_traces[:, idx]
            nonpos = np.where(line <= 0)[0]
            if nonpos.size > 0:
                cutoff = nonpos[0]
                if cutoff == 0:
                    continue
                x_vals = rounds_range[:cutoff]
                y_vals = line[:cutoff]
            else:
                x_vals = rounds_range
                y_vals = line
            ax_log.plot(x_vals, y_vals, color=cmap(idx % 20), alpha=0.6, linewidth=1)
            ax_linear.plot(x_vals, y_vals, color=cmap(idx % 20), alpha=0.6, linewidth=1)

        ax_log.set_title('Sample Country Wealth (log scale)', fontsize=14, fontweight='bold')
        ax_log.set_xlabel('Round', fontsize=12, fontweight='bold')
        ax_log.set_ylabel('Private Value (log)', fontsize=12, fontweight='bold')
        ax_log.set_yscale('log')
        ax_log.grid(True, alpha=0.3, which='both')

        ax_linear.set_title('Sample Country Wealth (linear)', fontsize=14, fontweight='bold')
        ax_linear.set_xlabel('Round', fontsize=12, fontweight='bold')
        ax_linear.set_ylabel('Private Value', fontsize=12, fontweight='bold')
        ax_linear.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        final_values = country_traces[-1]
        data = []
        for idx, cid in enumerate(country_order):
            wealth = final_values[idx]
            if wealth <= 0:
                continue
            stats = country_actions.get(cid, {'attacks': 0, 'bargains': 0})
            data.append({
                'Country ID': cid,
                'Final Wealth': wealth,
                'Attacks Attempted': stats.get('attacks', 0),
                'Bargains': stats.get('bargains', 0)
            })
        if data:
            df = pd.DataFrame(data).sort_values('Final Wealth', ascending=False).head(20)
            with pd.option_context('display.float_format', lambda x: f"{x:,.12g}"):
                display(df)
        else:
            display(pd.DataFrame(columns=['Country ID', 'Final Wealth', 'Attacks Attempted', 'Bargains']))



In [4]:
def make_slider(min_val, max_val, step, value, handle_color='#3B82F6'):
    slider = widgets.FloatSlider(min=min_val, max=max_val, step=step, value=value,
                                 readout=False, continuous_update=False,
                                 layout=widgets.Layout(width='220px'))
    slider.style.handle_color = handle_color
    slider.style.description_width = '0px'
    return slider

num_countries_slider = widgets.IntSlider(value=100, min=10, max=200, step=10, readout=False, continuous_update=False,
                                         layout=widgets.Layout(width='220px'))
num_countries_slider.style.handle_color = '#3B82F6'
num_rounds_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, readout=False, continuous_update=False,
                                      layout=widgets.Layout(width='220px'))
num_rounds_slider.style.handle_color = '#3B82F6'
mc_trials_slider = widgets.IntSlider(value=10, min=5, max=100, step=5, readout=False, continuous_update=False,
                                     layout=widgets.Layout(width='220px'))
mc_trials_slider.style.handle_color = '#3B82F6'

attack_cost_slider = make_slider(0.05, 0.30, 0.01, ds.ATTACK_COST_PERCENTAGE)
failed_cost_slider = make_slider(0.20, 0.70, 0.02, ds.FAILED_ATTACK_COST_PERCENTAGE)
defender_defense_loss_slider = make_slider(0.0, 0.20, 0.01, ds.DEFENDER_DEFENSE_LOSS_PERCENTAGE)
bargain_ev_slider = make_slider(0.02, 0.25, 0.01, ds.BARGAIN_EV_PERCENTAGE)

value_labels = {}


def decorate_slider(label_text, slider, fmt="{:.2f}"):
    value_labels[slider] = widgets.Label(fmt.format(slider.value), layout=widgets.Layout(width='60px'))

    def _update(change):
        value_labels[slider].value = fmt.format(change['new'])

    slider.observe(_update, names='value')
    header = widgets.HBox([
        widgets.Label(label_text, layout=widgets.Layout(width='120px', color='#374151')),
        value_labels[slider]
    ], layout=widgets.Layout(justify_content='space-between', width='220px'))
    return widgets.VBox([header, slider])

controls = {
    'num_countries': num_countries_slider,
    'num_rounds': num_rounds_slider,
    'attack_cost': attack_cost_slider,
    'failed_attack_cost': failed_cost_slider,
    'defender_defense_loss': defender_defense_loss_slider,
    'bargain_ev': bargain_ev_slider,
    'mc_trials': mc_trials_slider,
}

param_grid = widgets.GridBox(
    children=[
        decorate_slider('Countries', num_countries_slider, fmt="{}"),
        decorate_slider('Rounds', num_rounds_slider, fmt="{}"),
        decorate_slider('MC runs', mc_trials_slider, fmt="{}"),
        decorate_slider('Attack cost', attack_cost_slider),
        decorate_slider('Failed cost', failed_cost_slider),
        decorate_slider('Defender loss', defender_defense_loss_slider),
        decorate_slider('Bargain EV', bargain_ev_slider),
    ],
    layout=widgets.Layout(grid_template_columns='repeat(4, 1fr)', grid_row_gap='12px', grid_column_gap='18px', width='100%')
)

ui = widgets.VBox([
    widgets.HTML('<h4 style="margin-bottom:8px;">Simulation Parameters</h4>'),
    param_grid,
], layout=widgets.Layout(border='1px solid #ddd', padding='12px', border_radius='6px', width='100%'))

output = widgets.interactive_output(plot_totals, controls)

display(ui, output)



Output()